In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def _dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=_dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.countries,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

Post-loading: `Assignment`: 100%|██████████| 6/6 [00:00<00:00, 60.55it/s]


In [8]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db2 = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db2)

    stats = pstats.Stats(pr)

Post-loading: `Assignment`: 100%|██████████| 6/6 [00:00<00:00, 212.97it/s]


In [9]:
# db[User].extract()

In [10]:
db[User].keys()[0]

'16c7cfb1-a757-4ccf-b285-cbcf96ef50af'

In [11]:
del db[User][db[User].keys()[0]]

In [12]:
db[User].df()

_id,name,age
str,str,i64
"""1db6b635-e8d9-457d-80b5-1ab09e…","""Jack""",66
"""b0409880-13a4-4e81-bc67-856138…","""John""",26


In [13]:
db2[User].df()

_id,name,age
str,str,i64
"""066baca6-90f0-47d4-bbb2-343efa…","""Jack""",66
"""c22f2288-ec2a-4d9a-8f61-f5bda5…","""John""",31
"""e14ac4d6-3037-44cc-a725-6c92cc…","""Jane""",36


In [14]:
db3 = db | db2
db3.describe()


{
    'contents': {
        'records': {
            'schema.User': 5,
            'schema.Task': 12,
            'schema.Project': 3,
            'schema.Organization': 6,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 14},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 12, 'schema.Membership': 14}
    },
    'backend': None
}

In [15]:
x = db[Search.results]._link
assert x is not None
x.df()

schema.Search.term,schema.Search.results._rel._id,number,name,start,end,status,org__id
str,str,i64,str,date,date,str,str
"""test""","""104d65b581""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""51dcdcb7-c4d7-4f5c-af20-8386fb…"
"""test""","""11faab1a21""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""884b25cc-5974-4631-b3b9-794ad5…"
"""test""","""14941c52d3""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""b4c2e65b-b716-4568-a181-159a9d…"


In [16]:
x = db[Search.results.x.tasks.x.assignees]
print(x.select_str)
x.keys()
x.df(sort_by=[User.name])

SELECT DISTINCT "schema.Search".term AS "schema.Search.term",
                "schema.Search.results".number AS "schema.Search.results.number",
                "schema.Search.results.tasks"._id AS "schema.Search.results.tasks._id",
                "schema.Task.assignees"._id AS "schema.Task.assignees._id",
                "schema.Task.assignees"._id AS _id,
                "schema.Task.assignees".name AS name,
                "schema.Task.assignees".age AS age
FROM "schema_Search" AS "schema.Search"
JOIN "schema_SearchResult" AS "schema.Search.results._rel" ON "schema.Search.results._rel"._from_term = "schema.Search".term
JOIN "schema_Project" AS "schema.Search.results" ON "schema.Search.results._rel"._to_number = "schema.Search.results".number
JOIN "schema_Task" AS "schema.Search.results.tasks" ON "schema.Search.results.tasks".project_number = "schema.Search.results".number
JOIN "schema_Assignment" AS "schema.Search.results.tasks.assignees._rel" ON "schema.Search.results.tasks.assigne

schema.Search.term,schema.Search.results.number,schema.Search.results.tasks._id,_id,name,age
str,i64,str,str,str,i64
"""test""",0,"""665cf8b4-1509-4449-b4ed-eaa6e4…","""e14ac4d6-3037-44cc-a725-6c92cc…","""Jane""",36
"""test""",1,"""27ecf5ae-ce4b-4937-bd8b-65e73f…","""e14ac4d6-3037-44cc-a725-6c92cc…","""Jane""",36
"""test""",0,"""135f12d2-744d-480d-bce2-b236b0…","""b0409880-13a4-4e81-bc67-856138…","""John""",26
"""test""",2,"""74846d49-939e-49a5-bbce-1e6b89…","""b0409880-13a4-4e81-bc67-856138…","""John""",26
"""test""",1,"""6bc1994d-4c1c-413b-bd03-41ee73…","""b0409880-13a4-4e81-bc67-856138…","""John""",26
"""test""",0,"""f59fa840-3446-4774-91d8-1a7c8e…","""b0409880-13a4-4e81-bc67-856138…","""John""",26
"""test""",1,"""53517e35-42d3-45a7-b6cd-2e86b5…","""c22f2288-ec2a-4d9a-8f61-f5bda5…","""John""",31
"""test""",2,"""440f8854-426a-4130-a87b-e913db…","""c22f2288-ec2a-4d9a-8f61-f5bda5…","""John""",31
"""test""",0,"""957e3fa1-c00b-49c0-8cdc-8591e6…","""c22f2288-ec2a-4d9a-8f61-f5bda5…","""John""",31


In [17]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev152+g2c0bcb5.d20250123',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': 'd30f62a8d1f6a43e808b3c3f26ee3923b3e4c2e7',
        'module_path': WindowsPath('exp/schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'schema.User': 5,
            'schema.Task': 12,
            'schema.Project': 3,
            'schema.Organization': 6,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 14},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 12, 'schema.Membership': 14}
    },
    'backend': None
}

In [18]:
nodes, edges = db.to_graph([Search, Project, Organization])

In [19]:
nodes

,node_id,table,schema.Search.term,schema.Search.result_count,schema.Project.number,schema.Project.name,schema.Project.start,schema.Project.end,schema.Project.status,schema.Project.org__id,schema.Organization._id,schema.Organization.name,schema.Organization.address,schema.Organization.city
0,0,schema_Search,test,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,schema_Project,NaN,NaN,0.0,baking cake,2020-01-01,2020-01-04,done,b4c2e65b-b716-4568-a181-159a9d1276a0,NaN,NaN,NaN,NaN
2,2,schema_Project,NaN,NaN,1.0,cleaning shoes,2020-01-02,2020-01-05,done,884b25cc-5974-4631-b3b9-794ad5952b75,NaN,NaN,NaN,NaN
3,3,schema_Project,NaN,NaN,2.0,fixing cars,2020-01-03,2020-01-06,started,51dcdcb7-c4d7-4f5c-af20-8386fbb93ebd,NaN,NaN,NaN,NaN
4,4,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1010d3c8-c839-4969-a90d-247ea54cea32,Car Shop,Main Street 3,Cartown
5,5,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1e041f8b-4f3f-4b81-8070-97f007c52641,Shoe Shop,Main Street 2,Shoetown
6,6,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51dcdcb7-c4d7-4f5c-af20-8386fbb93ebd,Car Shop,Main Street 3,Cartown
7,7,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,884b25cc-5974-4631-b3b9-794ad5952b75,Shoe Shop,Main Street 2,Shoetown
8,8,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a2662a8b-6f2c-4b20-9d75-96e94c0e5183,Bakery,Main Street 1,Bakerville
9,9,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b4c2e65b-b716-4568-a181-159a9d1276a0,Bakery,Main Street 1,Bakerville


In [20]:
edges

,source,target,ltr,rtl,schema.SearchResult._to_number,schema.SearchResult._id,schema.SearchResult._from_term,schema.SearchResult.score
0,1,9,org__id,None,NaN,NaN,NaN,NaN
1,2,7,org__id,None,NaN,NaN,NaN,NaN
2,3,6,org__id,None,NaN,NaN,NaN,NaN
3,0,3,_to_number,_from_term,2.0,104d65b581,test,0.7
4,0,2,_to_number,_from_term,1.0,11faab1a21,test,0.8
5,0,1,_to_number,_from_term,0.0,14941c52d3,test,0.9


In [21]:
db[User].values()


[
    User({'_id': '066baca6-90f0-47d4-bbb2-343efa4c83b7', 'name': 'Jack', 'age': 66}),
    User({'_id': '1db6b635-e8d9-457d-80b5-1ab09e8b6509', 'name': 'Jack', 'age': 66}),
    User({'_id': 'b0409880-13a4-4e81-bc67-856138dc428a', 'name': 'John', 'age': 26}),
    User({'_id': 'c22f2288-ec2a-4d9a-8f61-f5bda51e0784', 'name': 'John', 'age': 31}),
    User({'_id': 'e14ac4d6-3037-44cc-a725-6c92cc45ab25', 'name': 'Jane', 'age': 36})
]

In [22]:
db[Project].df()

number,name,start,end,status,org__id
i64,str,date,date,str,str
0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""b4c2e65b-b716-4568-a181-159a9d…"
1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""884b25cc-5974-4631-b3b9-794ad5…"
2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""51dcdcb7-c4d7-4f5c-af20-8386fb…"


In [ ]:
db[Search.term].values()

['test']

In [24]:
list(db[Search.result_count].items())

[('test', 3)]

In [ ]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 0, 'b4c2e65b-b716-4568-a181-159a9d1276a0'),
        Organization({'_id': 'b4c2e65b-b716-4568-a181-159a9d1276a0', 'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville'})
    ),
    (
        ('test', 1, '884b25cc-5974-4631-b3b9-794ad5952b75'),
        Organization({'_id': '884b25cc-5974-4631-b3b9-794ad5952b75', 'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown'})
    ),
    (
        ('test', 2, '51dcdcb7-c4d7-4f5c-af20-8386fbb93ebd'),
        Organization({'_id': '51dcdcb7-c4d7-4f5c-af20-8386fbb93ebd', 'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown'})
    )
]

In [ ]:
db[Search.results].items()


[
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    ),
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    ),
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    )
]

In [27]:
list(db[Search.results]._abs_cols.keys())

['number', 'name', 'start', 'end', 'status', 'org__id']

In [28]:
db[Search.results].df()

schema.Search.term,number,name,start,end,status,org__id
str,i64,str,date,date,str,str
"""test""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""b4c2e65b-b716-4568-a181-159a9d…"
"""test""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""884b25cc-5974-4631-b3b9-794ad5…"
"""test""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""51dcdcb7-c4d7-4f5c-af20-8386fb…"


In [29]:
db[Search.results.x.org].df()

schema.Search.term,schema.Search.results.number,_id,name,address,city
str,i64,str,str,str,str
"""test""",0,"""b4c2e65b-b716-4568-a181-159a9d…","""Bakery""","""Main Street 1""","""Bakerville"""
"""test""",1,"""884b25cc-5974-4631-b3b9-794ad5…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""test""",2,"""51dcdcb7-c4d7-4f5c-af20-8386fb…","""Car Shop""","""Main Street 3""","""Cartown"""


In [30]:
db[Organization].df()

_id,name,address,city
str,str,str,str
"""1010d3c8-c839-4969-a90d-247ea5…","""Car Shop""","""Main Street 3""","""Cartown"""
"""1e041f8b-4f3f-4b81-8070-97f007…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""51dcdcb7-c4d7-4f5c-af20-8386fb…","""Car Shop""","""Main Street 3""","""Cartown"""
"""884b25cc-5974-4631-b3b9-794ad5…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""a2662a8b-6f2c-4b20-9d75-96e94c…","""Bakery""","""Main Street 1""","""Bakerville"""
"""b4c2e65b-b716-4568-a181-159a9d…","""Bakery""","""Main Street 1""","""Bakerville"""


In [ ]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_id,score,_from_term,_to_number
str,f64,str,i64
"""14941c52d3""",0.9,"""test""",0


In [ ]:
import pandas as pd


db[Organization.countries].df(pd.DataFrame)

value
schema.Organization._id              schema.Organization.countries.idx        
1010d3c8-c839-4969-a90d-247ea54cea32 0                                     USA
                                     1                                  Canada
1e041f8b-4f3f-4b81-8070-97f007c52641 0                                     USA
                                     1                                  Mexico
51dcdcb7-c4d7-4f5c-af20-8386fbb93ebd 0                                     USA
                                     1                                  Canada
884b25cc-5974-4631-b3b9-794ad5952b75 0                                     USA
                                     1                                  Mexico
a2662a8b-6f2c-4b20-9d75-96e94c0e5183 0                                     USA
                                     1                                  Mexico
                                     2                                  Canada
b4c2e65b-b716-4568-a181-159a9d1276a0 0                                     USA
                                     1                                  Mexico
                                     2                                  Canada

In [33]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         535248 function calls (481089 primitive calls) in 1.103 seconds

   Ordered by: cumulative time
   List reduced from 1779 to 156 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       18    0.000    0.000    0.462    0.026 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1570(__ior__)
       18    0.001    0.000    0.461    0.026 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3029(_mutate)
       11    0.000    0.000    0.455    0.041 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:850(load)
    36/12    0.001    0.000    0.436    0.036 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:664(_load_records)
       55    0.001    0.000    0.434    0.008 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:699(_load_rec_from_item)
       55    0.002    0.000    0.432    0.008 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.p

 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:941(df)
       18    0.000    0.000    0.226    0.013 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3003(_df_to_table)
19952/11732    0.026    0.000    0.222    0.000 C:\Users\lworm\Repos\py-research\src\py_research\caching.py:382(__get__)
      448    0.004    0.000    0.199    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:704(__hash__)
16598/481    0.023    0.000    0.171    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:54(inner)
16598/481    0.047    0.000    0.167    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:69(gen_int_hash)
 1903/507    0.002    0.000    0.163    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:43(_hash_sequence)
       18    0.003    0.000    0.160    0.009 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3259(_mutate_from_sql)
7006/4616    0.015    0.000    0.160    0.000 C:\Use